# Parsing Amber parameter file format
Why are we interested?
* We can use Amber's parmterization methods to generate non-bonded parameters and charges for non-standard residues and ligands.
* Example of how to do this: 
    * http://ambermd.org/tutorials/basic/tutorial5/index.htm 
    * http://ambermd.org/tutorials/basic/tutorial4b/index.htm
* We are currently in the process of setting up Amber on the GPU so that such workflows could be easily used by anyone.
* Assuming, we have parameterized a residue or in general some collaborator gave us an Amber parmeter file, how do we go from there to MCCE?

### What's the input format
* The input file has an extension `.prmtop` or `.parm7`
* Daunting at first... But it is really compact and full of information
* Let's see what does it look like, I will use example of a single ALA residue.



In [2]:
!cat amber_formats/ala5_gas.parm7

%VERSION  VERSION_STAMP = V0001.000  DATE = 04/21/14  14:22:50                  
%FLAG TITLE                                                                     
%FORMAT(20a4)                                                                   
NALA                                                                            
%FLAG POINTERS                                                                  
%FORMAT(10I8)                                                                   
      53       8      27      25      59      34     102      86       0       0
     270       5      25      34      86      12      23      20      11       0
       0       0       0       0       0       0       0       0      12       0
       0
%FLAG ATOM_NAME                                                                 
%FORMAT(20a4)                                                                   
N   H1  H2  H3  CA  HA  CB  HB1 HB2 HB3 C   O   N   H   CA  HA  CB  HB1 HB2 HB3 
C   O   N   H   CA 

* Obviously it contains way more information then we need (because it is meant for MD)


### How can we parse it
* We can write a parser ... No!
* We won't re-invent the wheel coz there are opensource tools already available that can do the job for us.
* Two are useful in particular
    * ParmEd - A cross-platform parameter editor (http://parmed.github.io/ParmEd/html/index.html)
    * MDTraj - A cross-platform trajectory editor (http://mdtraj.org)
* Let's use ParmEd to reterieve connectivity table, charges and non-bonded parameters
* If you don't have parmed, you can install it with a single command: `conda install parmed -c omnia`



In [5]:
import parmed as pmd
input_parms = pmd.load_file('amber_formats/ala5_gas.parm7')

* Connectivity data

In [13]:
for bond in input_parms.bonds:
    print(bond.atom1, bond.atom2)

<class 'parmed.amber._amberparm.AmberParm'>
(<Atom C [10]; In ALA 0>, <Atom O [11]; In ALA 0>)
(<Atom C [10]; In ALA 0>, <Atom N [12]; In ALA 1>)
(<Atom CA [4]; In ALA 0>, <Atom CB [6]; In ALA 0>)
(<Atom CA [4]; In ALA 0>, <Atom C [10]; In ALA 0>)
(<Atom N [0]; In ALA 0>, <Atom CA [4]; In ALA 0>)
(<Atom C [20]; In ALA 1>, <Atom O [21]; In ALA 1>)
(<Atom C [20]; In ALA 1>, <Atom N [22]; In ALA 2>)
(<Atom CA [14]; In ALA 1>, <Atom CB [16]; In ALA 1>)
(<Atom CA [14]; In ALA 1>, <Atom C [20]; In ALA 1>)
(<Atom N [12]; In ALA 1>, <Atom CA [14]; In ALA 1>)
(<Atom C [30]; In ALA 2>, <Atom O [31]; In ALA 2>)
(<Atom C [30]; In ALA 2>, <Atom N [32]; In ALA 3>)
(<Atom CA [24]; In ALA 2>, <Atom CB [26]; In ALA 2>)
(<Atom CA [24]; In ALA 2>, <Atom C [30]; In ALA 2>)
(<Atom N [22]; In ALA 2>, <Atom CA [24]; In ALA 2>)
(<Atom C [40]; In ALA 3>, <Atom O [41]; In ALA 3>)
(<Atom C [40]; In ALA 3>, <Atom N [42]; In ALA 4>)
(<Atom CA [34]; In ALA 3>, <Atom CB [36]; In ALA 3>)
(<Atom CA [34]; In ALA 3>, <A

* Non-bonded parameter data

In [15]:
for atom in input_parms.atoms:
    print(atom.name, atom.charge, atom.rmin, atom.epsilon)

('N', 0.1414, 1.8240000001431578, 0.16999999991766696)
('H1', 0.1997, 0.5999999999731258, 0.015700000004219245)
('H2', 0.1997, 0.5999999999731258, 0.015700000004219245)
('H3', 0.1997, 0.5999999999731258, 0.015700000004219245)
('CA', 0.0962, 1.9080000000152688, 0.10939999991572774)
('HA', 0.08889999999999999, 1.099999999494113, 0.015700000062726108)
('CB', -0.059699999999999996, 1.9080000000152688, 0.10939999991572774)
('HB1', 0.029999999999999995, 1.4869999997254757, 0.01570000002623629)
('HB2', 0.029999999999999995, 1.4869999997254757, 0.01570000002623629)
('HB3', 0.029999999999999995, 1.4869999997254757, 0.01570000002623629)
('C', 0.6163000005487781, 1.9079999997313366, 0.08600000012835884)
('O', -0.5722000021951126, 1.661200000277935, 0.20999999984182244)
('N', -0.4157, 1.8240000001431578, 0.16999999991766696)
('H', 0.27190000000000003, 0.5999999999731258, 0.015700000004219245)
('CA', 0.033699999999999994, 1.9080000000152688, 0.10939999991572774)
('HA', 0.0823, 1.3869999991698632, 0

### Why stop at Amber
* The real benefit of using ParmEd is that it is cross MD Platform (at least all major MD packages)
* What if someone gives a molecule, paramterized in charm or gromacs
* We repeat the exact same process as as above with the corresponding file format.
